In [1]:
import requests
import pandas as pd

from utils import state_names, state_names_short, state_alphas

In [1]:
usda_api_key = 'YOUR-API-KEY-HERE'
fred_api_key = 'YOUR-API-KEY-HERE'

## Crop yields

In [30]:
def get_crop(crop_info, states):
    """
    Downloads county-level yield data from USDA NASS API for 1979 - 2020.
    Input: crop_info = dictionary with API request information
           states = list of U.S. state abbreviations
    Output: pandas dataframe
    """
    # Retain only these columns
    retain_cols = ['Value', 'short_desc', 'year', 'county_code', 'state_fips_code']
    
    # Build API request
    request = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key + '&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&agg_level_desc=COUNTY&year__GE=1979&year__LE=2020'
    for key in crop_info:
        request += key + '=' + crop_info[key]
    
    # Dataframe for results
    df_out = pd.DataFrame()

    # Loop through states
    for state in states:
        data = requests.get(request + '&state_alpha=' + state)
        if data.status_code == 200:
            data = data.json()
            data = pd.DataFrame(data["data"])
            df_out = pd.concat([df_out, data], ignore_index=True)

    return df_out[retain_cols]

In [31]:
def tidy_yield_area(yield_in, area_in):
    """
    Merge and tidy rainfed yield and area data from USDA API
    """
    yields = yield_in.copy()
    areas = area_in.copy()
    
    # Yield tidy
    yields.rename(columns = {'Value':'yield', 'state_fips_code':'state'}, inplace=True)
    yields = yields[yields['county_code'] != '998']
    yields['fips'] = yields['state'] + yields['county_code']
    yields.drop(columns = ['county_code', 'short_desc'], inplace=True)
    yields['yield'] = yields['yield'].astype(float)
    
    # Area tidy
    areas.rename(columns = {'Value':'area', 'state_fips_code':'state'}, inplace=True)
    areas = areas[areas['county_code'] != '998']
    areas['fips'] = areas['state'] + areas['county_code']
    areas.drop(columns = ['county_code', 'short_desc'], inplace=True)
    areas['area'] = areas['area'].str.replace(',','').astype(float)
    
    # Merge
    return pd.merge(yields.drop_duplicates(subset=['fips','year']),
                    areas.drop_duplicates(subset=['fips','year']),
                    on = ['fips', 'year', 'state'],
                    how='inner')

In [23]:
%%time
############ Maize ##############
maize_yield_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE'}
maize_area_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'AREA HARVESTED'}
maize_yield = get_crop(maize_yield_request, state_names_short)
maize_area = get_crop(maize_area_request, state_names_short)

############## Soy ##############
soy_yield_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE'}
soy_area_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'AREA HARVESTED'}
soy_yield = get_crop(soy_yield_request, state_names_short)
soy_area = get_crop(soy_area_request, state_names_short)

CPU times: user 2.54 s, sys: 552 ms, total: 3.09 s
Wall time: 3min 10s


In [30]:
# Maize
print(maize_yield.shape, maize_area.shape)

maize_all = tidy_yield_area(maize_yield, maize_area)
print(maize_all.shape)

maize_all.to_csv('../input_data/usda_maize_yields_1979-2020.csv', index=False)

(69062, 5) (69006, 5)
(50769, 5)


In [31]:
# Soy
print(soy_yield.shape, soy_area.shape)

soy_all = tidy_yield_area(soy_yield, soy_area)
print(soy_all.shape)

soy_all.to_csv('../input_data/usda_soy_yields_1979-2020.csv', index=False)

(61053, 5) (61044, 5)
(46697, 5)


In [32]:
%%time
############# Maize rainfed vs irrigated ##############
maize_yield_nonirr_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE', '&prodn_practice_desc':'NON-IRRIGATED'}
maize_area_nonirr_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'AREA HARVESTED', '&prodn_practice_desc':'NON-IRRIGATED'}
maize_yield_nonirr = get_crop(maize_yield_nonirr_request, state_names_short)
maize_area_nonirr = get_crop(maize_area_nonirr_request, state_names_short)

maize_yield_irr_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE', '&prodn_practice_desc':'IRRIGATED'}
maize_area_irr_request = {'&commodity_desc':'CORN', '&util_practice_desc':'GRAIN', '&statisticcat_desc':'AREA HARVESTED', '&prodn_practice_desc':'IRRIGATED'}
maize_yield_irr = get_crop(maize_yield_irr_request, state_names_short)
maize_area_irr = get_crop(maize_area_irr_request, state_names_short)

# ############## Soy rainfed vs irrigated ##############
soy_yield_nonirr_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE', '&prodn_practice_desc':'NON-IRRIGATED'}
soy_area_nonirr_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'AREA HARVESTED', '&prodn_practice_desc':'NON-IRRIGATED'}
soy_yield_nonirr = get_crop(soy_yield_nonirr_request, state_names_short)
soy_area_nonirr = get_crop(soy_area_nonirr_request, state_names_short)

soy_yield_irr_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'YIELD', '&unit_desc':'BU / ACRE', '&prodn_practice_desc':'IRRIGATED'}
soy_area_irr_request = {'&commodity_desc':'SOYBEANS', '&statisticcat_desc':'AREA HARVESTED', '&prodn_practice_desc':'IRRIGATED'}
soy_yield_irr = get_crop(soy_yield_irr_request, state_names_short)
soy_area_irr = get_crop(soy_area_irr_request, state_names_short)

CPU times: user 4.47 s, sys: 1.01 s, total: 5.48 s
Wall time: 5min 47s


In [33]:
# Maize tidy
print(maize_yield_irr.shape, maize_area_irr.shape)

maize_irr_out = tidy_yield_area(maize_yield_irr, maize_area_irr)
print(maize_irr_out.shape)

print(maize_yield_nonirr.shape, maize_area_nonirr.shape)

maize_nonirr_out = tidy_yield_area(maize_yield_nonirr, maize_area_nonirr)
print(maize_nonirr_out.shape)

(8024, 5) (8013, 5)
(7298, 5)
(8422, 5) (8398, 5)
(7710, 5)


In [45]:
# Merge and store
maize_all = pd.merge(maize_irr_out, maize_nonirr_out, on=['fips', 'year', 'state'], suffixes=('_irr','_nonirr'), how='inner')
maize_all.to_csv('../input_data/usda_maize_yields_irr_nonirr_1979-2020.csv', index=False)

In [34]:
# Soy tidy
print(soy_yield_irr.shape, soy_area_irr.shape)

soy_irr_out = tidy_yield_area(soy_yield_irr, soy_area_irr)
print(soy_irr_out.shape)

print(soy_yield_nonirr.shape, soy_area_nonirr.shape)

soy_nonirr_out = tidy_yield_area(soy_yield_nonirr, soy_area_nonirr)
print(soy_nonirr_out.shape)

(5755, 5) (5755, 5)
(5473, 5)
(5861, 5) (5857, 5)
(5527, 5)


In [47]:
# Merge and store
soy_all = pd.merge(soy_irr_out, soy_nonirr_out, on=['fips', 'year', 'state'], suffixes=('_irr','_nonirr'), how='inner')
soy_all.to_csv('../input_data/usda_soy_yields_irr_nonirr_1979-2020.csv', index=False)

## Crop prices

In [3]:
def tidy_prices(prices_in):
    """
    Tidies national historical crop prices from USDA NASS API
    """
    # Output will be pandas dataframe
    prices_in = prices_in.json()
    df_out = pd.DataFrame(prices_in['data'])
    
    # Annual prices
    df_out = df_out[df_out['reference_period_desc'] == 'MARKETING YEAR']
    
    # Rename and select final columns
    df_out['value'] = df_out['Value'].apply(lambda x: float(x))
    df_out.rename(columns = {'value':'price', 'state_fips_code':'state'}, inplace=True)
    
    return df_out[['year', 'price']]

In [4]:
# Maize
maize_price = requests.get('http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key + '&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN' + 
                            '&statisticcat_desc=PRICE RECEIVED&unit_desc=$ / BU&agg_level_desc=NATIONAL&year__GE=1948&year__LE=2016')
maize_price = tidy_prices(maize_price)

# Soy 
soy_price = requests.get('http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key + '&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN' + 
                            '&statisticcat_desc=PRICE RECEIVED&unit_desc=$ / BU&agg_level_desc=NATIONAL&year__GE=1948&year__LE=2016')
soy_price = tidy_prices(soy_price)

In [110]:
############################
# Deflate with PPI data
############################

In [6]:
##### Producer Price Index by Commodity: Farm Products
##### Index 1982=100
##### Not Seasonally Adjusted 
##### https://fred.stlouisfed.org/series/WPU01

# HTTP request from STL FRED
data = requests.get('https://api.stlouisfed.org/fred/series/observations?series_id=WPU01&api_key=' + fred_api_key + '&file_type=json')
data = data.json()

# Pandas dataframe
ppi = pd.DataFrame(data['observations'])[['date','value']]
ppi['year'] = pd.to_datetime(ppi['date'])
ppi['value'] = ppi['value'].astype(float)

# We use annual prices
ppi_annual = ppi.set_index('year').resample('1Y').mean()
ppi_annual.rename(columns={'value':'ppi'}, inplace=True)
ppi_annual.index = ppi_annual.index.year

In [7]:
# Merge prices and adjust for inflation (set to 2018 USD)
ppi_2018 = ppi_annual.loc[2018]['ppi']

# Maize
maize_ppi = pd.merge(maize_price, ppi_annual.reset_index(), on='year', how='inner')
maize_ppi['price_ppi'] = maize_ppi['price'] / maize_ppi['ppi'] * ppi_2018
maize_ppi = maize_ppi.sort_values(by='year')

# Soy
soy_ppi = pd.merge(soy_price, ppi_annual.reset_index(), on='year', how='inner')
soy_ppi['price_ppi'] = soy_ppi['price'] / soy_ppi['ppi'] * ppi_2018
soy_ppi = soy_ppi.sort_values(by='year')

In [8]:
# Store
maize_ppi[['year', 'price', 'price_ppi']].to_csv('../input_data/usda_maize_prices_deflated_1948-2016.csv', index=False)
soy_ppi[['year', 'price', 'price_ppi']].to_csv('../input_data/usda_soy_prices_deflated_1948-2016.csv', index=False)

## Irrigation: Water applied (maize & soy)

In [197]:
def get_crop_water_applied(crop_name, states):
    """
    Downloads state-level irrigation data (water applied) from USDA NASS API for 2013, 2018.
    """
    # Retain only these columns
    retain_cols = ['Value', 'short_desc', 'year', 'state_fips_code', 'state_name']
    
    # Build API request
    request = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key + '&source_desc=CENSUS&sector_desc=CROPS&group_desc=FIELD CROPS&agg_level_desc=STATE&commodity_desc=' + crop_name + '&statisticcat_desc=WATER APPLIED&unit_desc=ACRE FEET / ACRE&domain_desc=TOTAL'
    
    # Dataframe for results
    df_out = pd.DataFrame()

    # Loop through states
    for state in states:
        data = requests.get(request + '&state_alpha=' + state)
        if data.status_code == 200:
            data = data.json()
            data = pd.DataFrame(data["data"])
            df_out = pd.concat([df_out, data], ignore_index=True)

    return df_out[retain_cols]

In [192]:
%%time
# Maize
maize_irr = get_crop_water_applied('CORN', state_names_short)

# Soy
soy_irr = get_crop_water_applied('SOYBEANS', state_names_short)

CPU times: user 750 ms, sys: 32.5 ms, total: 782 ms
Wall time: 13.1 s


In [193]:
#################
# Tidy and store
#################

retain_cols = ['year', 'state_fips_code', 'state_name', 'water_applied']

# Maize
maize_irr = maize_irr[maize_irr['short_desc'] == 'CORN, GRAIN, IRRIGATED - WATER APPLIED, MEASURED IN ACRE FEET / ACRE'] # subset grain for maize
maize_irr['Value'] = maize_irr['Value'].astype(float) * 304.8 # acre-feet/acre -> mm/acre
maize_irr.rename(columns = {'Value' : 'water_applied'}, inplace=True)
maize_irr[retain_cols].to_csv('../input_data/usda_maize_water_applied_2013-2018.csv', index=False)

# Soy
soy_irr['Value'] = soy_irr['Value'].astype(float) * 304.8 # acre-feet/acre -> mm/acre
soy_irr.rename(columns = {'Value' : 'water_applied'}, inplace=True)
soy_irr[retain_cols].to_csv('../input_data/usda_soy_water_applied_2013-2018.csv', index=False)

## Irrigation: Acres harvested

In [ ]:
def get_crop_irr_area(crop_name, states):
    """
    Downloads county-level irrigated acreage data from USDA NASS API
    """
    # Retain only these columns
    retain_cols = ['Value', 'short_desc', 'year', 'county_code', 'state_fips_code']
    
    # Build API request
    request = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key + '&source_desc=CENSUS&sector_desc=CROPS&group_desc=FIELD CROPS&agg_level_desc=COUNTY&commodity_desc=' + crop_name + '&prodn_practice_desc=IRRIGATED&statisticcat_desc=AREA HARVESTED'
    if crop_name == 'CORN':
        request += '&util_practice_desc=GRAIN'
    # Dataframe for results
    df_out = pd.DataFrame()

    # Loop through states
    for state in states:
        data = requests.get(request + '&state_alpha=' + state)
        if data.status_code == 200:
            data = data.json()
            data = pd.DataFrame(data["data"])
            df_out = pd.concat([df_out, data], ignore_index=True)

    return df_out[retain_cols]

In [ ]:
%%time
# Maize
maize_irr = get_crop_irr_area('CORN', state_names_short)

# Soy
soy_irr = get_crop_irr_area('SOYBEANS', state_names_short)

CPU times: user 1.4 s, sys: 342 ms, total: 1.75 s
Wall time: 1min 49s


In [11]:
def str_to_float_with_excp(x):
    try:
        return float(x)
    except:
        return 0.0
    
def tidy_irr_area(irr_in):
    """
    Tidy irrigated acreage data from USDA API
    """
    irr = irr_in.copy()
    
    # Yield tidy
    irr.rename(columns = {'Value':'irrigated_acreage', 'state_fips_code':'state'}, inplace=True)
    irr = irr[irr['county_code'] != '998']
    irr['fips'] = irr['state'] + irr['county_code']
    irr.drop(columns = ['county_code', 'short_desc'], inplace=True)
    irr['irrigated_acreage'] = irr['irrigated_acreage'].apply(lambda x: str_to_float_with_excp(x))
    
    return irr

In [17]:
# Maize
print(maize_irr.shape)

maize_irr_out = tidy_irr_area(maize_irr)
print(maize_irr_out.shape)

maize_irr_out.to_csv('../input_data/usda_maize_irrigated_acres_1997-2017.csv', index=False)

(8876, 5)
(8876, 4)


In [18]:
# Soy
print(soy_irr.shape)

soy_irr_out = tidy_irr_area(soy_irr)
print(soy_irr_out.shape)

soy_irr_out.to_csv('../input_data/usda_soy_irrigated_acres_1997-2017.csv', index=False)

(7610, 5)
(7610, 4)


## Irrigation: infrastructure expenses

In [181]:
def get_irr_expenses(states):
    """
    Downloads state-level new irrigation equipment costs from USDA NASS API
    """
    # Retain only these columns
    retain_cols = ['Value', 'short_desc', 'year', 'state_fips_code', 'state_name', 'domaincat_desc']
    
    # Build API request
    request = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key
    
    add_info = {'&source_desc':'CENSUS',
                '&sector_desc':'ECONOMICS',
                '&group_desc':'IRRIGATION',
                '&agg_level_desc':'STATE',
                '&commodity_desc':'FACILITIES %26 EQUIPMENT'}


    for key in add_info:
        request += key + '=' + add_info[key]

    # Dataframe for results
    df_out = pd.DataFrame()

    # Loop through states
    for state in states:
        data = requests.get(request + '&state_alpha=' + state)
        if data.status_code == 200:
            data = data.json()
            data = pd.DataFrame(data["data"])
            df_out = pd.concat([df_out, data], ignore_index=True)

    return df_out[retain_cols]

In [182]:
%%time
irr_expenses = get_irr_expenses(state_names_short)

CPU times: user 534 ms, sys: 28.5 ms, total: 563 ms
Wall time: 15.6 s


### Expenses associated with new expansion

In [263]:
# Filter new expansion
irr_expenses_new = irr_expenses[irr_expenses['domaincat_desc'] == 'EXPENSE, PRIMARY PURPOSE: (NEW EXPANSION)']

# The category 'land clearing & levelling for new irrigation acreage' is classified differently so add manually:
land_clearing_new_acres = irr_expenses[(irr_expenses['short_desc'] == 'FACILITIES & EQUIPMENT, IRRIGATION, LAND CLEARING & LEVELING FOR NEW IRRIGATION ACREAGE - ACRES IRRIGATED')
                                       & (irr_expenses['domaincat_desc'] == 'EXPENSE: (ANY)')]

land_clearing_new_expense = irr_expenses[(irr_expenses['short_desc'] == 'FACILITIES & EQUIPMENT, IRRIGATION, LAND CLEARING & LEVELING FOR NEW IRRIGATION ACREAGE - EXPENSE, MEASURED IN $')]

irr_expenses_new = pd.concat([irr_expenses_new, land_clearing_new_acres, land_clearing_new_expense])

In [264]:
# We want expenses per acre for each category
def get_categories(x):
    # Remove extraneous details
    out =  x.replace('FACILITIES & EQUIPMENT, IRRIGATION', '').replace(' - ', '').replace('ACRES IRRIGATED', '').replace('EXPENSE, MEASURED IN $', '').replace(', ', '')
    # There is one row that includes the average over all
    if out == '':
        out = 'ALL'
    return out

def get_units(x):
    # Remove extraneous details
    return x[-15:].replace(', ', '')

irr_expenses_new['category'] = irr_expenses_new['short_desc'].apply(lambda x: get_categories(x))
irr_expenses_new['units'] = irr_expenses_new['short_desc'].apply(lambda x: get_units(x))

In [265]:
# Tidy
def str_to_float(x):
    if type(x) == str:
        return float(x.replace(',', '').replace('-', 'NaN').replace('(D)', 'NaN').rstrip())
    else:
        return x
    
    
irr_expenses_new = irr_expenses_new[['Value', 'year', 'state_fips_code', 'category', 'units']]
irr_expenses_new['Value'] = irr_expenses_new['Value'].apply(str_to_float)

In [266]:
# Get expenses per acre and sum 
irr_expenses_new = pd.pivot(irr_expenses_new.set_index(['state_fips_code', 'year', 'category']), columns='units', values='Value').reset_index()
irr_expenses_new = irr_expenses_new.groupby(['state_fips_code', 'year', 'category']).apply(lambda x: x['MEASURED IN $'] / x['ACRES IRRIGATED'])
irr_expenses_new = pd.DataFrame({'cost_per_acre' : irr_expenses_new}).reset_index()
irr_expenses_new = irr_expenses_new[irr_expenses_new['category'] != 'ALL'].groupby(['state_fips_code', 'year']).sum().drop(columns='level_3')

In [267]:
# Store
irr_expenses_new.to_csv('../input_data/usda_irrigation_expansion_cost_2013-2018.csv')

### Expenses associated with maintenance

In [201]:
# to do
irr_expenses_maint = irr_expenses[irr_expenses['domaincat_desc'] == 'EXPENSE, PRIMARY PURPOSE: (SCHEDULED REPLACEMENT OR MAINTENANCE)']

## Irrigation: application expenses

In [16]:
def get_irrapp_expenses(states):
    """
    Downloads state-level new irrigation equipment costs from USDA NASS API
    """
    # Retain only these columns
    retain_cols = ['Value', 'short_desc', 'year', 'state_fips_code', 'state_name', 'domaincat_desc']
    
    # Build API request
    request = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' + usda_api_key
    
    add_info = {'&source_desc':'CENSUS',
                '&sector_desc':'ECONOMICS',
                '&group_desc':'IRRIGATION',
                '&agg_level_desc':'STATE',
                '&commodity_desc':'ENERGY', 
                '&statisticcat_desc':'EXPENSE'}

    for key in add_info:
        request += key + '=' + add_info[key]

    # Dataframe for results
    df_out = pd.DataFrame()

    # Loop through states
    for state in states:
        data = requests.get(request + '&state_alpha=' + state)
        if data.status_code == 200:
            data = data.json()
            data = pd.DataFrame(data["data"])
            df_out = pd.concat([df_out, data], ignore_index=True)

    return df_out[retain_cols]

In [24]:
%%time
irrapp_expenses = get_irrapp_expenses(state_names_short)

CPU times: user 480 ms, sys: 77.6 ms, total: 558 ms
Wall time: 6.64 s


In [25]:
# Subset to cost per acre when using electrcity (dominant energy source)
irrapp_expenses = irrapp_expenses[irrapp_expenses['short_desc'] == 'ENERGY, IRRIGATION, ON FARM PUMPING, ELECTRICITY, IN THE OPEN ONLY - EXPENSE, MEASURED IN $ / ACRE IRRIGATED']

# Subset to groundwater only
irrapp_expenses = irrapp_expenses[irrapp_expenses['domaincat_desc'] == 'WATER SOURCE: (GROUND ONLY)']

In [26]:
# Tidy
def str_to_float(x):
    if type(x) == str:
        return float(x.replace(',', '').replace('-', 'NaN').replace('(D)', 'NaN').rstrip())
    else:
        return x
    
    
irrapp_expenses = irrapp_expenses[['Value', 'year', 'state_fips_code', 'state_name']].copy()
irrapp_expenses['Value'] = irrapp_expenses['Value'].apply(str_to_float)
irrapp_expenses.rename(columns = {'Value' : 'irr_app_cost_per_acre'}, inplace=True)

In [27]:
# Store
irrapp_expenses.to_csv('../input_data/usda_irrigation_application_cost_per_acre_2013-2018.csv', index=False)

## Irrigation: water applied (all crops)

In [7]:
# Read in USDA data downloaded from 00c_usda_fris_wget.sh
df_2013 = pd.read_csv('../input_data/usda_water_applied_2013.csv', sep=',', skiprows=3, header=1)
df_2018 = pd.read_csv('../input_data/usda_water_applied_2018.csv', sep=',', skiprows=3, header=1)

In [8]:
# Select relevant column: Average acre-feet applied from groundwater
df_2018 = df_2018[['Unnamed: 2', 'Ground water from wells.5']].copy()
df_2018 = df_2018.rename(columns = {'Unnamed: 2':'state_name', 'Ground water from wells.5': 'water_applied_per_acre'})

df_2013 = df_2013[['Unnamed: 2', 'Ground water from wells.5']].copy()
df_2013 = df_2013.rename(columns = {'Unnamed: 2':'state_name', 'Ground water from wells.5': 'water_applied_per_acre'})

# Subset Midwest
df_2018_mw = df_2018[df_2018['state_name'].isin(state_names)]
df_2018_mw.insert(0, 'year', 2018)

df_2013_mw = df_2013[df_2013['state_name'].isin(state_names)]
df_2013_mw.insert(0, 'year', 2013)

In [9]:
# Join and store
df_out = pd.concat([df_2018_mw, df_2013_mw])
df_out['water_applied_per_acre'] = df_out['water_applied_per_acre'].astype(float) * 304.8 # feet -> mm

df_out.to_csv('../input_data/usda_water_applied_2013-2018_tidy.csv')